# Music Store Analysis

### Please note that you can see some data different from the video because the provided data is bit different from which used in the video. you can se the difference clearly if you run the average command separately from moderate question number 3.

# Connecting jupyter notebook to database

In [1]:
import mysql.connector

In [2]:
data_base = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "123456",
    database = "music_store"
)
mycursor = data_base.cursor()

# Question Set 1 - Easy

## 1. Who is the senior most employee based on job title?

In [3]:
mycursor.execute("SELECT CONCAT(first_name,' ',last_name) FROM music_store.employee ORDER BY levels DESC LIMIT 1")

In [4]:
for x in mycursor:
    print(x)

('Andrew Adams',)


## 2. Which countries have the most Invoices?

In [5]:
mycursor.execute("""
                    SELECT billing_country, COUNT(*) AS total_count FROM music_store.invoice
                    GROUP BY billing_country
                    ORDER BY total_count DESC 
                    LIMIT 1
                    """)

In [6]:
for x in mycursor:
    print(x)

('USA', 131)


## 3. What are top 3 values of total invoice?

In [7]:
mycursor.execute("""
                    SELECT * FROM music_store.invoice
                    ORDER BY total DESC
                    LIMIT 3
                    """)

In [8]:
for x in mycursor:
    print(x)

(183, 42, '2018-02-09 00:00:00', '9, Place Louis Barthou', 'Bordeaux', 'None', 'France', '33000', 23.759999999999998)
(92, 32, '2017-07-02 00:00:00', '696 Osborne Street', 'Winnipeg', 'MB', 'Canada', 'R3L 2B9', 19.8)
(526, 5, '2020-06-08 00:00:00', 'Klanova 9/506', 'Prague', 'None', 'Czech Republic', '14700', 19.8)


## 4. Which city has the best customers? We would like to throw a promotional Music Festival in the city we made the most money. Write a query that returns one city that has the highest sum of invoice totals. Return both the city name & sum of all invoice totals

In [9]:
mycursor.execute("""
                    SELECT billing_country, SUM(total) AS total_invoice FROM music_store.invoice
                    GROUP BY billing_country
                    ORDER BY total_invoice DESC 
                    LIMIT 1
                    """)

In [10]:
for x in mycursor:
    print(x)

('USA', 1040.4899999999998)


## 5. Who is the best customer? The customer who has spent the most money will be declared the best customer. Write a query that returns the person who has spent the most money

In [11]:
mycursor.execute("""
                    SELECT customer.customer_id,first_name , last_name, SUM(invoice.total) AS total_spending FROM music_store.customer
                    JOIN music_store.invoice 
                    ON customer.customer_id = invoice.customer_id
                    GROUP BY customer.customer_id, first_name , last_name
                    ORDER BY total_spending DESC
                    LIMIT 1
                    """)

In [12]:
for x in mycursor:
    print(x)

(5, 'FrantiÅ¡ek', 'WichterlovÃ¡', 144.54000000000002)


# Question Set 2 - Moderate

## Q1: Write query to return the email, first name, last name, & Genre of all Rock Music listeners. Return your list ordered alphabetically by email starting with A.

In [13]:
mycursor.execute("""
                    SELECT DISTINCT email,first_name,last_name FROM music_store.customer
                    JOIN music_store.invoice ON customer.customer_id = invoice.customer_id
                    JOIN music_store.invoice_line ON invoice.invoice_id = invoice_line.invoice_id
                    WHERE track_id IN (
                     SELECT track.track_id FROM music_store.track
                     JOIN music_store.genre ON track.genre_id = genre.genre_id
                     WHERE genre.name LIKE "Rock"
                    )
                    ORDER BY email;
                    """)

In [14]:
for x in mycursor:
    print(x)

('aaronmitchell@yahoo.ca', 'Aaron', 'Mitchell')
('alero@uol.com.br', 'Alexandre', 'Rocha')
('astrid.gruber@apple.at', 'Astrid', 'Gruber')
('bjorn.hansen@yahoo.no', 'BjÃ¸rn', 'Hansen')
('camille.bernard@yahoo.fr', 'Camille', 'Bernard')
('daan_peeters@apple.be', 'Daan', 'Peeters')
('diego.gutierrez@yahoo.ar', 'Diego', 'GutiÃ©rrez')
('dmiller@comcast.com', 'Dan', 'Miller')
('dominiquelefebvre@gmail.com', 'Dominique', 'Lefebvre')
('edfrancis@yachoo.ca', 'Edward', 'Francis')
('eduardo@woodstock.com.br', 'Eduardo', 'Martins')
('ellie.sullivan@shaw.ca', 'Ellie', 'Sullivan')
('emma_jones@hotmail.com', 'Emma', 'Jones')
('enrique_munoz@yahoo.es', 'Enrique', 'MuÃ±oz')
('fernadaramos4@uol.com.br', 'Fernanda', 'Ramos')
('fharris@google.com', 'Frank', 'Harris')
('fralston@gmail.com', 'Frank', 'Ralston')
('frantisekw@jetbrains.com', 'FrantiÅ¡ek', 'WichterlovÃ¡')
('ftremblay@gmail.com', 'FranÃ§ois', 'Tremblay')
('fzimmermann@yahoo.de', 'Fynn', 'Zimmermann')
('hannah.schneider@yahoo.de', 'Hannah', 'Sch

## Q2: Let's invite the artists who have written the most rock music in our dataset. Write a query that returns the Artist name and total track count of the top 10 rock bands.

In [15]:
mycursor.execute("""
                    SELECT artist.artist_id, artist.name,COUNT(artist.artist_id) AS number_of_songs FROM track
                    JOIN album ON album.album_id = track.album_id
                    JOIN artist ON artist.artist_id = album.artist_id
                    JOIN genre ON genre.genre_id = track.genre_id
                    WHERE genre.name LIKE 'Rock'
                    GROUP BY artist.artist_id ,  artist.name
                    ORDER BY number_of_songs DESC
                    LIMIT 10;
                    """)

In [16]:
for x in mycursor:
    print(x)

(1, 'AC/DC', 18)
(3, 'Aerosmith', 15)
(8, 'Audioslave', 14)
(22, 'Led Zeppelin', 14)
(4, 'Alanis Morissette', 13)
(5, 'Alice In Chains', 12)
(23, 'Frank Zappa & Captain Beefheart', 9)
(2, 'Accept', 4)


## Q3: Return all the track names that have a song length longer than the average song length. Return the Name and Milliseconds for each track. Order by the song length with the longest songs listed first.

In [17]:
mycursor.execute("""
                    SELECT name , milliseconds FROM music_store.track
                    WHERE milliseconds > (
                     SELECT AVG(milliseconds) FROM music_store.track
                    )
                    ORDER BY milliseconds DESC;
                    """)

In [18]:
for x in mycursor:
    print(x)

('How Many More Times', 711836)
('Advance Romance', 677694)
('Sleeping Village', 644571)
('You Shook Me(2)', 619467)
("Talkin' 'Bout Women Obviously", 589531)
('Stratus', 582086)
('No More Tears', 555075)
('The Alchemist', 509413)
('Wheels Of Confusion / The Straightener', 494524)
('Book Of Thel', 494393)
('You Oughta Know (Alternate)', 491885)
('Terra', 482429)
("Snoopy's search-Red baron", 456071)
('Sozinho (Hitmakers Classic Mix)', 436636)
('Master Of Puppets', 436453)
('Stone Crazy', 433397)
('Snowblind', 420022)
('Computadores Fazem Arte', 404323)
('Jerusalem', 402390)
('Dazed and Confused', 401920)
('The Winner Loses', 392254)
('Love, Hate, Love', 387134)
('ConstruÃ§Ã£o / Deus Lhe Pague', 383059)
('Black Sabbath', 382066)
("Livin' On The Edge", 381231)
("I Can't Quit You Baby(2)", 380551)
('Princess of the Dawn', 375418)
('Harvester Of Sorrow', 374543)
('Whole Lotta Love', 373394)
("Momma's Gotta Die Tonight", 371539)
('Vai Passar', 369763)
('Wherever I May Roam', 369345)
('Overd